In [2]:
%cd drive/MyDrive/HackUTDProject/


/content/drive/MyDrive/HackUTDProject


In [3]:
!pip install twelvedata[pandas,matplotlib,plotly,websocket-client]



     |████████████████████████████████| 44 kB 1.4 MB/s 


In [13]:

table=pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
df = table[0]
df.to_csv('S&P500-Info.csv')
df.to_csv("S&P500-Symbols.csv", columns=['Symbol'])


In [5]:
import pandas as pd
from twelvedata import TDClient
import time
def get_usdata(symbol_name, interval_time, data_points,keynum):
    apikey1 = "9369e3dfc0e94eac916f5dd74d0fcadb"
    apikey2 = "2e16a63f1cc74165bb76b539d9714bf0"
    apikey3 = "9af0a3239ffd46e5be312d0944c45f70"
    apikey4 = "6ce628b88c56419d8e476125b0d164d1"
    apikey5 = "061d6abefa0043adaf2e7c9546065aad"
    apikey6 = "23042f079cd54ae48a2ebae53aeb8142"
    if keynum == 1:
        apikeyput = apikey1
    elif keynum == 2:
        apikeyput = apikey2
    elif keynum == 3:
        apikeyput = apikey3
    elif keynum == 4:
        apikeyput = apikey4
    elif keynum == 5:
        apikeyput = apikey5
    else:
        apikeyput = apikey6

    td = TDClient(apikey=apikeyput)
    # Construct the necessary time serie
    ts = td.time_series(
        symbol=symbol_name,
        interval=interval_time,
        outputsize=data_points
        )

    # Returns pandas.DataFrame
    data = ts.as_pandas()
    #data = data.sort_index(axis=1, ascending=True)
    data = data['close']
    data = data.iloc[::-1]
    
    #data.reset_index(inplace=True, drop=True)

    #print(data)
    return data
  
def main_pull(csvfile):
  starttime=time.time()
  data = pd.read_csv(csvfile)
  key = 1
  for index,row in data.iterrows():
    try:
      if((index+1)%12==0 and key<5):
          key=key+1
          df = get_usdata(row['Symbol'],'1day',365,key)
          df.to_csv(row['Symbol']+'.csv')
      elif((index+1)%12==0 and key>=5):
          time.sleep(90.0 - ((time.time() - starttime) % 60.0))
          key = 1
          starttime = time.time()
          df = get_usdata(row['Symbol'],'1day',365,key)
          df.to_csv(row['Symbol']+'.csv')
      else:
          df = get_usdata(row['Symbol'],'1day',365,key)
          df.to_csv(row['Symbol']+'.csv')
    except:
      print ("No out")
  

main_pull('S&P500-Info.csv')

No out


In [9]:
import pandas as pd

df = pd.read_csv('S&P500-Info.csv')
li = []
for index,row in df.iterrows():
  try:
    data = pd.read_csv(row['Symbol']+'.csv')
    annual = 100*(data.loc[data.index[-1], 'close']-data['close'][0])/data['close'][0]
    li.append(annual)
  except:
    li.append(-1)
df['Annual Return'] = li
df.to_csv('Symbols-info-annual.csv')

In [ ]:
import re
import tweepy
from tweepy import OAuthHandler
from textblob import TextBlob

class TwitterClient(object):
	'''
	Generic Twitter Class for sentiment analysis.
	'''
	def __init__(self):
		'''
		Class constructor or initialization method.
		'''
		# keys and tokens from the Twitter Dev Console
		consumer_key = ''
		consumer_secret = ''
		access_token = ''
		access_token_secret = ''

		# attempt authentication
		try:
			# create OAuthHandler object
			self.auth = OAuthHandler(consumer_key, consumer_secret)
			# set access token and secret
			self.auth.set_access_token(access_token, access_token_secret)
			# create tweepy API object to fetch tweets
			self.api = tweepy.API(self.auth)
		except:
			print("Error: Authentication Failed")

	def clean_tweet(self, tweet):
		'''
		Utility function to clean tweet text by removing links, special characters
		using simple regex statements.
		'''
		return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])
									|(\w+:\/\/\S+)", " ", tweet).split())

	def get_tweet_sentiment(self, tweet):
		'''
		Utility function to classify sentiment of passed tweet
		using textblob's sentiment method
		'''
		# create TextBlob object of passed tweet text
		analysis = TextBlob(self.clean_tweet(tweet))
		# set sentiment
		if analysis.sentiment.polarity > 0:
			return 'positive'
		elif analysis.sentiment.polarity == 0:
			return 'neutral'
		else:
			return 'negative'

	def get_tweets(self, query, count = 10):
		'''
		Main function to fetch tweets and parse them.
		'''
		# empty list to store parsed tweets
		tweets = []

		try:
			# call twitter api to fetch tweets
			fetched_tweets = self.api.search(q = query, count = count)

			# parsing tweets one by one
			for tweet in fetched_tweets:
				# empty dictionary to store required params of a tweet
				parsed_tweet = {}

				# saving text of tweet
				parsed_tweet['text'] = tweet.text
				# saving sentiment of tweet
				parsed_tweet['sentiment'] = self.get_tweet_sentiment(tweet.text)

				# appending parsed tweet to tweets list
				if tweet.retweet_count > 0:
					# if tweet has retweets, ensure that it is appended only once
					if parsed_tweet not in tweets:
						tweets.append(parsed_tweet)
				else:
					tweets.append(parsed_tweet)

			# return parsed tweets
			return tweets

		except tweepy.TweepError as e:
			# print error (if any)
			print("Error : " + str(e))

def main():
	# creating object of TwitterClient Class
	api = TwitterClient()
	# calling function to get tweets
	tweets = api.get_tweets(query = 'Donald Trump', count = 200)

	# picking positive tweets from tweets
	ptweets = [tweet for tweet in tweets if tweet['sentiment'] == 'positive']
	# percentage of positive tweets
	print("Positive tweets percentage: {} %".format(100*len(ptweets)/len(tweets)))
	# picking negative tweets from tweets
	ntweets = [tweet for tweet in tweets if tweet['sentiment'] == 'negative']
	# percentage of negative tweets
	print("Negative tweets percentage: {} %".format(100*len(ntweets)/len(tweets)))
	# percentage of neutral tweets
	print("Neutral tweets percentage: {} % \
		".format(100*(len(tweets) -(len( ntweets )+len( ptweets)))/len(tweets)))

	# printing first 5 positive tweets
	print("\n\nPositive tweets:")
	for tweet in ptweets[:10]:
		print(tweet['text'])

	# printing first 5 negative tweets
	print("\n\nNegative tweets:")
	for tweet in ntweets[:10]:
		print(tweet['text'])

if __name__ == "__main__":
	# calling main function
	main()


In [1]:
!pip install pytrends


In [ ]:

from pytrends.request import TrendReq
import pandas as pd
import time
df = pd.read_csv('S&P500-Info.csv')

for index, rows in df.iterrows():
    kw_list =[rows['Security']] 
    pytrends.build_payload(kw_list , cat=0, timeframe='today 12-m')
    data = pytrends.interest_over_time() 
    related =pytrends.related_queries()
    try:
      new_data = related[rows['Security']]['rising'].head(3)
      for i,r in new_data.iterrows():
        try:
          li = [r['query']] 
          pytrends.build_payload(li , cat=0, timeframe='today 12-m')
          re = pytrends.interest_over_time()
          data[li[0]] = re[li[0]]
        except:
          print("error")
      try:
        data = data.drop('isPartial',1)
        data.to_csv(rows['Symbol']+'searchTrend'+'.csv')
      except:
        print("error")
    except:
      print("error")
  